# Optimización de Tiempo de Estudio para Trabajos Prácticos

Clase del día 17/08/2023

## Problema

Tengo 2 trabajos prácticos que me vencen en 4 horas. Un TP de Investigación Operativa, y uno de derecho. 
Quiero maximizar la suma de las notas de los prácticos, pero asegurándome de aprobar ambos prácticos.
Para el trabajo práctico de IO, en promedio sumo 30 puntos por cada hora que le dedico. Para el de derecho, sumo en promedio 40 puntos por cada hora que le dedico.
En ambas materias se aprueba con 60, y la máxima nota es 100.
Cuánto tiempo le debería dedicar a cada materia?


#### Importamos las librerías

In [1]:
from ortools.linear_solver import pywraplp

Definimos el Solver

Creamos el Solver. Para modelos de programación lineal no entera, el solver GLOP es el más eficiente.

In [2]:
solver = pywraplp.Solver.CreateSolver('GLOP')

## Variables

#### Definimos las variables

$$\text{Horas IO: Horas dedicadas al trabajo de Investigación Operativa.}$$
$$\text{Horas Der: Horas dedicadas al trabajo de Derecho.}$$


Para crear las variables en Python, utilizaremos la función NumVar, formada en el objeto Solver.

La sintáxis es: <br>
variable = solver.NumVar(min_value, max_value, name)<br>
Parámetros:<br>
- min_value: El valor mínimo que puede tomar la variable. Puede ser cualquier número real o -solver.infinity() si no hay límite inferior.
- max_value: El valor máximo que puede tomar la variable. Puede ser cualquier número real o solver.infinity() si no hay límite superior.
- name: Nombre opcional para la variable. Puede servir para depurar errores y para interepretación de las soluciones.

In [3]:
horas_io = solver.NumVar(0, solver.infinity(), 'Horas dedicadas al trabajo de Investigación Operativa') 
horas_der = solver.NumVar(0, solver.infinity(), 'Horas dedicadas al trabajo de Derecho')

Atributos de la función NumVar:<br>
- lb(): Devuelve el límite inferior de la variable. (Esto ya lo establecimos en la celda anterior)
- ub(): Devuelve el límite superior de la variable. (Esto ya lo establecimos en la celda anterior)
- name(): Devuelve el nombre de la variable. (Esto ya lo establecimos en la celda anterior)
- solution_value(): Devuelve el valor de la variable en la solución óptima. Esto lo podemos hacer una vez que se haya resuelto el problema solamente.

In [4]:
horas_io.name()

'Horas dedicadas al trabajo de Investigación Operativa'

In [5]:
horas_io.lb()

0.0

In [6]:
horas_io.ub()

inf

## Restricciones

Las restricciones son las siguientes:
$$
\begin{align*}
\quad & 30 \cdot \text{HorasIO} \geq 60 \\
& 40 \cdot \text{HorasDer} \geq 60 \\
& 30 \cdot \text{HorasIO} \leq 100 \\
& 40 \cdot \text{HorasDer} \leq 100 \\
& \text{HorasIO} + \text{HorasDer} \leq 3 \\
& \text{HorasIO} \geq 0 
,  \text{HorasDer} \geq 0
\end{align*}
$$

In [7]:
# Mínima nota para aprobar IO
rest_aprobar_io = solver.Add(30 * horas_io >= 60, 'Mínima nota para aprobar IO')

# Mínima nota para aprobar Derecho
rest_aprobar_der = solver.Add(40 * horas_der >= 60, 'Mínima nota para aprobar Derecho')

# Máxima nota posible en IO
rest_max_nota_io = solver.Add(30 * horas_io <= 100, 'Máxima nota posible en IO')

# Máxima nota posible en Derecho
rest_max_nota_der = solver.Add(40 * horas_der <= 100, 'Máxima nota posible en Derecho')

# 4 horas de tiempo disponible
rest_tiempo = solver.Add(horas_io + horas_der <= 4, 'Tiempo disponible')

# Las restricciones de no negatividad ya están puestas en la definición de variables

Atributos:
- restriccion.name(): Nombre de la restricción.
- restriccion.lb(): Valor mínimo que puede tomar el RHS.
- restriccion.ub(): Valor máximo que puede tomar el RHS.
- restriccion.GetCoefficient(variable): Ver cuál es el valor del coeficiente que acompaña a tal variable.

In [8]:
rest_aprobar_io.name()

'Mínima nota para aprobar IO'

In [9]:
rest_aprobar_io.GetCoefficient(horas_io)

30.0

In [10]:
rest_aprobar_io.lb()

60.0

In [11]:
rest_aprobar_io.ub()

inf

## Función Objetivo


$$
\text{Maximizar} \quad  30 \cdot \text{HorasIO} + 40 \cdot \text{HorasDer} 
$$

In [12]:
solver.Maximize(30 * horas_io + 40 * horas_der)

## Resolver el problema

In [13]:
status = solver.Solve()

## Resultados

In [14]:
if status == pywraplp.Solver.OPTIMAL:
    print('El problema tiene una solución óptima')
else:
    print('No se encontró una solución óptima')

El problema tiene una solución óptima


In [15]:
horas_optimas_io = horas_io.solution_value()
horas_optimas_der = horas_der.solution_value()
suma_notas = solver.Objective().Value()
nota_io = horas_optimas_io * 30
nota_der = horas_optimas_der * 40

print(f'La cantidad de horas óptimas de IO son {horas_optimas_io}')
print(f'La cantidad de horas óptimas de Der son {horas_optimas_der}')
print(f'La nota de IO será {nota_io}')
print(f'La nota de Derecho será {nota_der}')
print(f'La suma de notas será {suma_notas}')

La cantidad de horas óptimas de IO son 2.0
La cantidad de horas óptimas de Der son 2.0
La nota de IO será 60.0
La nota de Derecho será 80.0
La suma de notas será 140.0


## Para resolver:

Y si ahora ya no me interesa obtener la máxima suma de notas, sino que quiero minimizar la cantidad de tiempo haciendo los prácticos. Cuánto tiempo debería dedicarle a cada materia?

In [16]:
solver.Minimize(horas_io + horas_der)
status = solver.Solve()

In [18]:
horas_io.solution_value()

2.0

In [17]:
horas_der.solution_value()


1.5